#### create net framework
---------
- input  : 'data' , params{'shape':(3,10,10)}

- layer_1: 'conv' , - params{'input_channel':3,'output_channel':8,'kernel_size':3,'pad':1,'stride':1,'group':1}

- layer_2: 'batchnorm' , params{}

- layer_3: 'relu'

- layer_4: 'conv_dw', 
params{'input_channel':8,'output_channel':8,'kernel_size':3,'pad':1,'stride':1,'group':8}

- layer_5: 'conv_dw_batchnorm', params{}

- layer_6: 'conv_dw_relu'

- layer_7: 'conv', 
params{'input_channel':8,'output_channel':16,'kernel_size':1,'pad':1,'stride':1,'group':1}

In [13]:
import logging
import mxnet as mx
import logging 
import numpy as np
from mxnet import autograd
from mxnet import ndarray as nd
from mxnet import gluon
from mxnet import image
import mxnet.gluon.nn as nn

In [2]:
logging.getLogger().setLevel(logging.INFO)

choose device

In [3]:
try:
    ctx = mx.gpu()
    _ = nd.array((1,),ctx=ctx)
except:
    ctx = mx.cpu()

#### Define the network
-------

##### prepare data

Q : train_iter and test_iter must have the same batch_size ?

In [12]:
data_set = nd.random.normal(shape=(10,3,20,20))

train_iter = mx.io.NDArrayIter(data=data_set, batch_size=1)

test_iter = mx.io.NDArrayIter(data=data_set[1:2,:,:,:], batch_size=1)

network

In [5]:
data = mx.sym.Variable('data')

conv1 = mx.sym.Convolution(data, name='conv1', num_filter=2, kernel=(3,3), pad=1, no_bias=True)

conv1_batchnorm = mx.sym.BatchNorm(conv1, name='conv1_batchnorm', fix_gamma=True)

conv1_relu = mx.sym.Activation(conv1_batchnorm, act_type='relu', name='conv1_relu')

conv2_dw =mx.sym.Convolution(conv1_relu, name='conv2_dw', num_filter=2, kernel=(3,3), pad=1, no_bias=True, num_group=2)

conv2_dw_batchnorm = mx.sym.BatchNorm(conv2_dw, name='conv2_dw_batchnorm', fix_gamma=True)

conv2_dw_relu = mx.sym.Activation(conv2_dw_batchnorm, act_type='relu', name='conv2_dw_relu')

conv2 = mx.sym.Convolution(conv2_dw_batchnorm, name='conv2', num_filter=2, kernel=(1,1), pad=0, no_bias=True)

flatten = mx.sym.flatten(data=conv2)

fc = mx.sym.FullyConnected(data=flatten, num_hidden=4)

net = mx.sym.SoftmaxOutput(data=fc, name='softmax')

In [14]:
mx.viz.plot_network(net)
mx.viz.

Fontconfig error: "/usr/X11/lib/X11/fontconfig/conf.d/10-scale-bitmap-fonts.conf", line 72: non-double matrix element
Fontconfig error: "/usr/X11/lib/X11/fontconfig/conf.d/10-scale-bitmap-fonts.conf", line 72: non-double matrix element
Fontconfig warning: "/usr/X11/lib/X11/fontconfig/conf.d/10-scale-bitmap-fonts.conf", line 80: saw unknown, expected number


CalledProcessError: Command '['dot', '-Tsvg']' returned non-zero exit status -11

In [6]:
model = mx.mod.Module(symbol=net, context=ctx)

bind data_shape

In [7]:
model.bind(data_shapes=train_iter.provide_data)

/Library/Python/2.7/site-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [8]:
model.init_params()

In [9]:
model.predict(test_iter)


[[0.25000003 0.25       0.24999999 0.24999996]]
<NDArray 1x4 @cpu(0)>

##### fuck, success finally

#### save model and load pretrained model
----

In [ ]:
model.save_checkpoint